In [1]:
import pandas
recomender_csv=pandas.read_csv("https://drive.google.com/uc?id=1HXoQQqsTIpQkG93onNjGxvV4VLET7NyT&export=download")
recomender_df=pandas.DataFrame(recomender_csv)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505176 sha256=60534e55563c20622164b0b069846e27a5c1e76a3a8e3edc040604b31ceee0ee
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [ ]:

from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy


# Prepare the data for collaborative filtering
reader = Reader(rating_scale=(1, 5))
collab_data = Dataset.load_from_df(recomender_df[['UserId', 'AttractionId', 'Rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(collab_data, test_size=0.2)

# Train the SVD model
svd = SVD()
svd.fit(trainset)

# Evaluate the model
predictions = svd.test(testset)
accuracy.rmse(predictions)

RMSE: 0.9125


0.9124914351042801

In [ ]:
#Generate recommendations for a user based on their cluster and predicted ratings:

# Function to get recommendations for a user based on cluster
def get_cluster_recommendations(user_id, model, data, top_n=10):
    user_cluster = data[data['UserId'] == user_id]['Cluster'].values[0]
    similar_users = data[data['Cluster'] == user_cluster]['UserId'].unique()
    all_attractions = data['AttractionId'].unique()
    user_attractions = data[data['UserId'] == user_id]['AttractionId'].unique()
    recommendations = []

    for attraction_id in all_attractions:
        if attraction_id not in user_attractions:
            est_rating = model.predict(user_id, attraction_id).est
            recommendations.append((attraction_id, est_rating))

    recommendations.sort(key=lambda x: x[1], reverse=True)
    top_recommendations = [attraction_id for attraction_id, rating in recommendations[:top_n]]

    # Print the names of the recommended attractions
    for rec_id in top_recommendations:
        attraction = data.loc[data['AttractionId'] == rec_id]
        if not attraction.empty:
            print(attraction['Attraction'].values[0])

    return top_recommendations

In [ ]:
recomendations=get_cluster_recommendations(67315,svd,recomender_df)


Bromo Tengger Semeru National Park
Waterbom Bali
Mount Semeru Volcano
Khayangan Reflexology & Massage
Jomblang Cave
Ullen Sentalu Museum
Jodipan Colorful Village
Merapi Volcano
Sempu Island
Uluwatu Temple


In [ ]:
# Recomendation system using Cosine-similarity
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
# Create user-item matrix
user_item_matrix = recomender_df.pivot_table(index='UserId', columns='AttractionId', values='Rating').fillna(0)

# Compute cosine similarity between users
cosine_sim = cosine_similarity(user_item_matrix)
cosine_sim_df = pd.DataFrame(cosine_sim, index=user_item_matrix.index, columns=user_item_matrix.index)

# Function to get recommendations
def get_recommendations(user_id, cosine_sim_df, user_item_matrix, data, top_n=10):
    similar_users = cosine_sim_df[user_id].sort_values(ascending=False).index[1:top_n+1]
    recommendations = user_item_matrix.loc[similar_users].mean(axis=0).sort_values(ascending=False)
    recommended_attractions = recommendations.index[:top_n]

    # Print the names of the recommended attractions
    for rec_id in recommended_attractions:
        attraction_names = []
        attraction_name = data.loc[data['AttractionId'] == rec_id, 'Attraction'].values[0]
        if attraction_name not in attraction_names:
            attraction_names.append(attraction_name)
        print(attraction_name)

    return attraction_name
recomendations=get_recommendations(67315,cosine_sim_df,user_item_matrix,recomender_df)
print(recomendations)

Nusa Dua Beach
Kuta Beach - Bali
Water Castle (Tamansari)
Ullen Sentalu Museum
Sewu Temple
Ratu Boko Temple
Ramayana Ballet at Prambanan
Merapi Volcano
Malioboro Road
Kalibiru National Park
Kalibiru National Park


In [2]:
#Recomendation system using Pearson's correlation

user_item_matrix = recomender_df.pivot_table(index='UserId', columns='AttractionId', values='Rating').fillna(0)
# Compute Pearson's correlation between users
pearson_corr = user_item_matrix.T.corr()
# Function to get recommendations
def get_recommendations(user_id, pearson_corr, user_item_matrix, data, top_n=10):
    similar_users = pearson_corr[user_id].sort_values(ascending=False).index[1:top_n+1]
    recommendations = user_item_matrix.loc[similar_users].mean(axis=0).sort_values(ascending=False)
    recommended_attractions = recommendations.index[:top_n]

    # Print the names of the recommended attractions
    attraction_names = []
    for rec_id in recommended_attractions:
        attraction_name = data.loc[data['AttractionId'] == rec_id, 'Attraction'].values[0]
        if attraction_name not in attraction_names:
            attraction_names.append(attraction_name)
        print(attraction_names)
    return attraction_names

# Get recommendations for UserId 1
recommendations = get_recommendations(67315, pearson_corr, user_item_matrix, recomender_df)



['Nusa Dua Beach']
['Nusa Dua Beach', 'Kuta Beach - Bali']
['Nusa Dua Beach', 'Kuta Beach - Bali', 'Water Castle (Tamansari)']
['Nusa Dua Beach', 'Kuta Beach - Bali', 'Water Castle (Tamansari)', 'Ullen Sentalu Museum']
['Nusa Dua Beach', 'Kuta Beach - Bali', 'Water Castle (Tamansari)', 'Ullen Sentalu Museum', 'Sewu Temple']
['Nusa Dua Beach', 'Kuta Beach - Bali', 'Water Castle (Tamansari)', 'Ullen Sentalu Museum', 'Sewu Temple', 'Ratu Boko Temple']
['Nusa Dua Beach', 'Kuta Beach - Bali', 'Water Castle (Tamansari)', 'Ullen Sentalu Museum', 'Sewu Temple', 'Ratu Boko Temple', 'Ramayana Ballet at Prambanan']
['Nusa Dua Beach', 'Kuta Beach - Bali', 'Water Castle (Tamansari)', 'Ullen Sentalu Museum', 'Sewu Temple', 'Ratu Boko Temple', 'Ramayana Ballet at Prambanan', 'Merapi Volcano']
['Nusa Dua Beach', 'Kuta Beach - Bali', 'Water Castle (Tamansari)', 'Ullen Sentalu Museum', 'Sewu Temple', 'Ratu Boko Temple', 'Ramayana Ballet at Prambanan', 'Merapi Volcano', 'Malioboro Road']
['Nusa Dua Beach